Ustvarimo tabelo z leti in semestri

In [1]:
semestri = []

for i in range(16,25):
    leto = "FRI " + "20" + str(i) + "/20" + str(i + 1)
    semestri.append(leto + ", zimski semester")
    semestri.append(leto + ", letni semester")
semestri


['FRI 2016/2017, zimski semester',
 'FRI 2016/2017, letni semester',
 'FRI 2017/2018, zimski semester',
 'FRI 2017/2018, letni semester',
 'FRI 2018/2019, zimski semester',
 'FRI 2018/2019, letni semester',
 'FRI 2019/2020, zimski semester',
 'FRI 2019/2020, letni semester',
 'FRI 2020/2021, zimski semester',
 'FRI 2020/2021, letni semester',
 'FRI 2021/2022, zimski semester',
 'FRI 2021/2022, letni semester',
 'FRI 2022/2023, zimski semester',
 'FRI 2022/2023, letni semester',
 'FRI 2023/2024, zimski semester',
 'FRI 2023/2024, letni semester',
 'FRI 2024/2025, zimski semester',
 'FRI 2024/2025, letni semester']

Uvozimo knjižnice

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Nastavitve za headless način
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Omogoči nevidno delovanje
options.add_argument("--disable-gpu")  # Prepreči težave na nekaterih sistemih
options.add_argument("--window-size=1920,1080")  # Nastavi standardno velikost okna
options.add_argument("--no-sandbox")  # Prepreči težave v nekaterih okoljih
options.add_argument("--disable-dev-shm-usage")  # Prepreči težave s pomnilnikom

driver = webdriver.Chrome(options=options)

Pridobimo podatke o predmetih v različnih letih in semestrih

In [3]:
BASE_URL = "https://urnik.fri.uni-lj.si/timetable" 
all_data = []

driver = webdriver.Chrome()
driver.get(BASE_URL)

for semester in semestri:
    try:
        # Klikni na semester
        link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, semester))
        )
        link.click()

        # Počakaj, da se naloži pravi <div> s predmeti
        predmeti_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[@style="height:500px;width:330px;overflow:auto"]'))
        )

        # Pridobi vse <a> elemente samo znotraj tega <div>
        predmeti_links = predmeti_div.find_elements(By.TAG_NAME, 'a')

        # Pobereš samo prave predmete
        #imena = [link.text.split('(')[0].strip() for link in predmeti_links]
        imena = [link.text for link in predmeti_links]


        # Shrani v all_data
        all_data.extend((semester, ime) for ime in imena)

        # Vrni se nazaj
        driver.back()
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, semester))
        )

    except Exception as e:
        print(f"Napaka pri semestru {semester}: {e}")

driver.quit()

# Izpis pridobljenih podatkov
all_data

Napaka pri semestru FRI 2017/2018, zimski semester: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6A91F1F55+78133]
	GetHandleVerifier [0x00007FF6A91F1FB0+78224]
	(No symbol) [0x00007FF6A8FB91BA]
	(No symbol) [0x00007FF6A900F19D]
	(No symbol) [0x00007FF6A900F44C]
	(No symbol) [0x00007FF6A90623D7]
	(No symbol) [0x00007FF6A903719F]
	(No symbol) [0x00007FF6A905F21F]
	(No symbol) [0x00007FF6A9036F33]
	(No symbol) [0x00007FF6A9000358]
	(No symbol) [0x00007FF6A90010C3]
	GetHandleVerifier [0x00007FF6A94BBA8D+3001453]
	GetHandleVerifier [0x00007FF6A94B5E72+2977874]
	GetHandleVerifier [0x00007FF6A94D497D+3103581]
	GetHandleVerifier [0x00007FF6A920C7EA+186826]
	GetHandleVerifier [0x00007FF6A92143FF+218591]
	GetHandleVerifier [0x00007FF6A91F9D94+110452]
	GetHandleVerifier [0x00007FF6A91F9F42+110882]
	GetHandleVerifier [0x00007FF6A91E0379+5465]
	BaseThreadInitThunk [0x00007FFDEDFBE8D7+23]
	RtlUserThreadStart [0x00007FFDEF7FBF6C+44]

Napaka pri semestru FRI 2017/2018, letni semester: Message: 


[('FRI 2016/2017, zimski semester',
  'Aktualno raziskovalno področje I (63545A)'),
 ('FRI 2016/2017, zimski semester',
  'Aktualno raziskovalno področje I (63546F)'),
 ('FRI 2016/2017, zimski semester',
  'Algoritmi in podatkovne strukture 1 (63711)'),
 ('FRI 2016/2017, zimski semester',
  'Algoritmi in podatkovne strukture 1 (63279)'),
 ('FRI 2016/2017, zimski semester',
  'Analiza algoritmov in hevristično reševanje problemov (63263)'),
 ('FRI 2016/2017, zimski semester', 'Angleški jezik nivo B (63223)'),
 ('FRI 2016/2017, zimski semester', 'Angleški jezik nivo B (63746)'),
 ('FRI 2016/2017, zimski semester', 'Digitalna vezja (63718)'),
 ('FRI 2016/2017, zimski semester', 'Digitalno načrtovanje (63260)'),
 ('FRI 2016/2017, zimski semester', 'Digitalno procesiranje signalov (63516)'),
 ('FRI 2016/2017, zimski semester', 'Digitalno procesiranje signalov (63744)'),
 ('FRI 2016/2017, zimski semester', 'Diskretne strukture (63203)'),
 ('FRI 2016/2017, zimski semester', 'Diskretne struktu

Ustvarimo pandas DataFrame

In [4]:
def pretvori_semester(semester):
    leto1 = semester[6:8]
    leto2 =  semester[11:13]
    if "zimski" in semester:
        sem = 1
    elif "letni" in semester:
        sem = 2
    return(str(leto1) + "/" + str(leto2) + "/" + str(sem))

predelani_podatki = []

for semester, predmet in all_data:
    leto = pretvori_semester(semester)  # Pokliči funkcijo za semester
    predelani_podatki.append((leto, predmet))  # Dodaj rezultat v seznam

    

predmeti_df = pd.DataFrame(predelani_podatki, columns=["semester", "Predmet"])

predmeti_df

,semester,Predmet
0,16/17/1,Aktualno raziskovalno področje I (63545A)
1,16/17/1,Aktualno raziskovalno področje I (63546F)
2,16/17/1,Algoritmi in podatkovne strukture 1 (63711)
3,16/17/1,Algoritmi in podatkovne strukture 1 (63279)
4,16/17/1,Analiza algoritmov in hevristično reševanje pr...
...,...,...
1536,24/25/2,Varnost programov (63774)
1537,24/25/2,Velepodatki (63560)
1538,24/25/2,Vhodno-izhodne naprave (63728)
1539,24/25/2,Visoko zmogljivo računanje (63543)
